In [ ]:
from os.path import dirname, abspath
from owlready2 import default_world, onto_path
from pandas import DataFrame, Series, to_datetime
from matplotlib.pyplot import subplots, title, show
from numpy import *
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
CURRENT_PATH = os.path.dirname(os.path.abspath(__file__))
DATABASE_PATH = CURRENT_PATH + "/4WD_car_db.sqlite3"
ONTO_IRI = "http://www.semanticweb.org/fenrir/ontologies/2023/5/4WD_car_specific_ontology#"

def get_onto():
    default_world.set_backend(filename=DATABASE_PATH, exclusive=False)
    onto_path.append(CURRENT_PATH)
    return default_world.get_ontology(CURRENT_PATH+ONTO_IRI).load()

def close():
    default_world.close()

onto = get_onto()

In [ ]:
battery_df = DataFrame(columns=["timestamp","voltage"])

for car in onto.search(is_a = onto.car):
    for battery in car.hasPart:
        if not isinstance(battery, onto.battery): continue
        for measure in battery.hasMeasure:
            battery_df.loc[len(battery_df)] = Series(dtype=object)
            for prop in measure.get_properties():
                for value in prop[measure]:
                    if prop.python_name == 'charge' : continue
                    battery_df.loc[len(battery_df)-1, prop.python_name] = value
print(battery_df.shape)

In [ ]:
battery_df.head()

In [ ]:
THRESHOLD = 600

battery_df['diff_time'] = battery_df['timestamp'].diff()
battery_df['cycle'] = (battery_df['diff_time'] > THRESHOLD).cumsum()
battery_df['timestamp'] = battery_df['timestamp'] - battery_df.groupby('cycle')['timestamp'].transform('min')
discharge_cycles = battery_df.groupby('cycle')

fig, ax = subplots()

for n, cycle in discharge_cycles:
    ax.plot(cycle.timestamp, cycle.voltage)
    pass

ax.set_xlabel('Timestamp')
ax.set_ylabel('Voltage')
title('Discharge patterns')

show()
discharge_cycles.describe()

In [ ]:
raw_df = DataFrame(columns=["timestamp"])

for assembly in onto.search(is_a = onto.Assembly):
    for part in assembly.hasPart:
        for measure in part.hasMeasure:
            raw_df.loc[len(raw_df)] = Series(dtype=object)
            for prop in measure.get_properties():
                for value in prop[measure]:
                    if prop.python_name == 'charge' : continue
                    raw_df.loc[len(raw_df)-1, prop.python_name] = value
raw_df = raw_df.drop('position', axis=1)
df = raw_df.copy()
df['timestamp'] = to_datetime(df['timestamp'], unit='s')
df['speed'] = df['speed'].apply(abs)
df = df.groupby('timestamp').sum()
df.head()

In [ ]:
close()